In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'files/SES-cold3-C4- 5DoF inputs_template.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:9,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:9,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PDI1 -109.089755  -4.477723  21.386613 -0.941514  10.804298
PDI2  111.429511  -3.828453  21.664246 -0.884323 -10.784784
PDI3   39.795278  52.011229  11.044961  4.741583  -3.798262
PDI4  -38.810833  51.903508  11.309534  4.728098   3.961031
PR1     0.943594   1.280695  -0.338386  0.031453   0.035137
PR2  -109.505749 -45.867074  22.370475 -4.911732  10.680266
PR3   111.309055 -45.373337  22.058458 -4.857077 -10.600321
PR4   103.076801  59.765562  30.696044  5.307985 -10.311998
PR5   -101.43058   59.40225  30.809096  5.269783  10.410043 

Provided 5DOF encoder values: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PDI1   -109.062609  -4.434582  21.454053 -0.933014  10.805269
PDI2    111.310127  -3.744227  21.815125 -0.889186 -10.782564
PDI3     39.730784

### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [5]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
Name,,,,,,,,,
PDI1,-109.089755,-4.477723,21.386613,-0.941514,10.804298,yellow,242.214916,264.860490,1241.274628
PDI2,111.429511,-3.828453,21.664246,-0.884323,-10.784784,yellow,-241.782625,263.638476,1241.619009
PDI3,39.795278,52.011229,11.044961,4.741583,-3.798262,yellow,-85.594929,142.863551,1281.345137
PDI4,-38.810833,51.903508,11.309534,4.728098,3.961031,yellow,89.257250,143.137547,1281.064648
PR1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
PR2,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
PR3,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
PR4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
PR5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [6]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('files/FDPR shifts - sandbox.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                                ...                           
PR1                     25                 27  244.0  ...     12.0     -117      120
PR3                     -1                  0  238.9  ...     17.1     -130      171
PR4                     37                 45  234.1  ...     21.9      -99      219
PR5                     -4                 -3  240.9  ...     15.1     -161      151
PR2                     62                 64  229.1  ...     26.9     -181      269

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [7]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team

In [8]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)

Residuals (in sMPA frame) baseline: 
      endpt_X  endpt_Y  endpt_Z
PR1    0.117   -0.120   -0.027
PR2    0.181   -0.269   -0.064
PR3    0.130   -0.171    0.000
PR4    0.099   -0.219   -0.045
PR5    0.161   -0.151    0.003 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.1051 -0.239   0.0635 -0.0027  0.0015] 

Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.021    0.065   -0.000
PR2    0.044   -0.085   -0.026
PR3   -0.007    0.015    0.026
PR4   -0.039   -0.030   -0.029
PR5    0.024    0.035    0.030 



### 1.3 Convert new best-fit poses to encoder space

In [9]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
                X          Y          Z        Rx         Ry
PDI1 -108.957623  -4.673383  21.517673 -0.935703  10.806732
PDI2  111.415113  -3.983029  21.878745 -0.891876 -10.781101
PDI3    39.83577  51.797818  11.236752  4.738663  -3.801113
PDI4  -38.718441  51.675813  11.472641  4.729882   3.959033
PR1     1.022114   1.076265  -0.221742  0.030651   0.037361
PR2  -109.372602 -46.037321  22.479731 -4.907139  10.684671
PR3   111.296014 -45.503057  22.249574  -4.86567 -10.594721
PR4   103.057297  59.582837  30.969257    5.3009 -10.314328
PR5  -101.314536  59.181725  31.006073  5.275549  10.406777 

Baseline pose encoders (displayed again here just for reference): 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PDI1   -109.062609  -4.434582  21.454053 -0.933014  10.805269
PDI2    111.310127  -3.744227  21.815125 -0.889186 -10.7

# 2 Write the new poses to an Excel file in the current directory

In [10]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from FDPR update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)

**Writing to Excel complete**
